# Predicción del abandono de empleados sobre los datos de competition
### Paulo Álvarez Da Costa. NIA: 100475757

Usaré el dataset de employee_competition proporcionado para predecir si un empleado abandonará la empresa o no.

#### Configuraciones iniciales

In [26]:
# Carga librerias necesarias
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Datos usados para la predicción
employees_comp = pd.read_csv("DATA/employee_competition.csv")

#### Limpieza y preprocesamiento de datos en competition

Voy a comenzar realizando el mismo proceso de limpieza y preprocesamiento que en el notebook anterior, para garantizar que el modelo pueda transformar correctamente las columnas del dataset 'competition', ya que el pipeline del modelo espera exactamente las mismas variables (nombres, tipos y orden). Si este paso no se realizara, el modelo no podría predecir los datos de 'competition', ya que no se le ha enseñado a transformar las columnas de este dataset. Por lo tanto, es fundamental que el preprocesamiento sea idéntico al del dataset 'train' para asegurar la compatibilidad y precisión del modelo en las predicciones.

In [27]:
# Eliminar las mismas columnas que se eliminaron en el notebook1 
employees_comp.drop(columns=['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeID'], inplace=True)

# Identificar columnas categóricas y numéricas
categorical_columns = employees_comp.select_dtypes(include=["object"]).columns.tolist()
numeric_columns = employees_comp.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Pipeline para transformar las variables categóricas
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputa valores nulos con la moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Transforma variables categóricas a numéricas
])

# Pipeline para transformar las variables numéricas
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputa valores nulos con la mediana
    ('scaler', RobustScaler())                      # Escala las variables numéricas
])

# Combinar ambos pipelines en un ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_columns),
    ('cat', categorical_pipeline, categorical_columns)
])

# Aplicar el preprocesamiento a las features
employees_comp_preprocessed = preprocessor.fit_transform(employees_comp)

print("Forma de los datos preprocesados:", employees_comp_preprocessed.shape)

Forma de los datos preprocesados: (1470, 46)


Como se puede observar, el dataset preprocesado presenta 46 columnas, las mismas que el dataset de entrenamiento, lo que garantiza que el modelo pueda realizar predicciones sin problemas.

#### Predicción final



In [29]:
# Cargar el modelo final
modelo_final = joblib.load("modelo_final.pkl")

# Realizar predicciones (0 = No, 1 = Yes)
predicciones_binarias = modelo_final.predict(employees_comp_preprocessed)

# Mapear a 'Yes' y 'No'
predicciones = pd.Series(predicciones_binarias).map({1: 'Yes', 0: 'No'})

# Guardar predicciones en CSV
df_resultado = pd.DataFrame({'Attrition': predicciones})
df_resultado.to_csv("predicciones.csv", index=False)

print("Predicciones guardadas en 'predicciones.csv'")


Predicciones guardadas en 'predicciones.csv'
